<a href="https://colab.research.google.com/github/jisoolee11/AI_study/blob/main/computer_vision(test2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> DACON 제 2회 컴퓨터 비전 학습 경진대회

In [7]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!unzip -qq /content/gdrive/My\ Drive/data_2/mnist_data.zip
!unzip -qq /content/gdrive/MyDrive/data_2/test_dirty_mnist.zip
!unzip -qq /content/gdrive/MyDrive/data_2/dirty_mnist.zip

In [9]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from torchvision import transforms
from torchvision.models import resnet50

In [10]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [11]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [12]:
trainset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/dirty_mnist_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32, num_workers=2)
test_loader = DataLoader(testset, batch_size=10, num_workers=1)

In [13]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model,(3, 256, 256)))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256

In [17]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.27614, 0.87019
0: 0.25656, 0.87740
0: 0.27526, 0.88221
0: 0.26616, 0.88942
0: 0.23910, 0.89663
0: 0.25285, 0.89063
0: 0.21490, 0.90986
0: 0.24371, 0.89063
0: 0.23662, 0.90625
0: 0.23923, 0.90024
0: 0.21814, 0.90024
0: 0.23477, 0.89543
0: 0.20143, 0.91466
0: 0.25522, 0.88101
0: 0.23983, 0.89543
1: 0.17990, 0.92788
1: 0.20773, 0.90385
1: 0.17562, 0.92067
1: 0.19765, 0.93269
1: 0.15743, 0.93029
1: 0.19304, 0.91587
1: 0.13384, 0.95072
1: 0.16481, 0.92909
1: 0.13643, 0.94832
1: 0.16671, 0.93510
1: 0.15019, 0.93510
1: 0.13735, 0.94591
1: 0.15632, 0.93870
1: 0.15235, 0.93990
1: 0.15353, 0.94111
2: 0.11931, 0.94712
2: 0.13610, 0.94111
2: 0.15356, 0.94111
2: 0.13689, 0.94832
2: 0.09863, 0.96514
2: 0.11763, 0.94591
2: 0.12391, 0.95433
2: 0.13778, 0.95313
2: 0.09855, 0.96635
2: 0.12404, 0.95553
2: 0.09672, 0.96514
2: 0.12253, 0.95072
2: 0.12591, 0.95793
2: 0.15920, 0.94231
2: 0.08119, 0.96755
3: 0.07718, 0.97476
3: 0.14162, 0.95313
3: 0.08627, 0.96154
3: 0.09116, 0.97236
3: 0.07763, 0.97115


In [16]:
submit = pd.read_csv('/content/gdrive/MyDrive/data_2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('submit1.csv', index=False)
from google.colab import files
files.download('submit1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>